# The Latin Square Problem

The latin square problem involves filling an $n \times n$ grid with $n$ different symbols, one symbol per cell, such that each symbol appears exactly once in each row and exactly once in each column. 

<img src="img/latin.png" alt="Latin Square" style="width:400px; height:150px;">

## Setup
We will begin by activating the environment and importing the necessary packages.

In [1]:
using Revise
using Pkg
Pkg.activate("../../../")
using SeaPearl

  Activating project at `c:\Users\leobo\Desktop\École\Poly\SeaPearl\SeaPearlZoo.jl`


## Modeling the Problem

We will now build a model for the problem

In [10]:
# Define the problem size
square_size = 3
matrix = zeros(Int64, (square_size, square_size))

trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
# model.limit.numberOfSolutions = limit

N = size(matrix)[1]
puzzle = Matrix{SeaPearl.AbstractIntVar}(undef, N, N)
for i = 1:N
    for j in 1:N
        puzzle[i, j] = SeaPearl.IntVar(1, N, "puzzle_" * string(i) * ", " * string(j), trailer)
        SeaPearl.addVariable!(model, puzzle[i, j]; branchable=true)
        if matrix[i, j] > 0
            push!(
                model.constraints,
                SeaPearl.EqualConstant(puzzle[i, j], matrix[i, j], trailer)
            )
        end
    end
end
for i in 1:N
    push!(model.constraints, SeaPearl.AllDifferent(puzzle[i, :], model.trailer))
    push!(model.constraints, SeaPearl.AllDifferent(puzzle[:, i], model.trailer))
end


## Solving the Problem

The model is built; let's solve it!

In [11]:
variableSelection = SeaPearl.MinDomainVariableSelection{false}()
valueSelection = SeaPearl.BasicHeuristic()

@time SeaPearl.solve!(model; variableHeuristic=variableSelection, valueSelection=valueSelection)

  8.226456 seconds (14.57 M allocations: 756.050 MiB, 3.16% gc time, 99.84% compilation time)


:Optimal

## Visualizing the Solution

Let's visualize the solution obtained

In [11]:
function get_best_solution(model::SeaPearl.CPModel)
    best_solution = nothing
    best_objective = Inf
    for solution in model.statistics.solutions
        if !isnothing(solution) && solution["totalValue"] < best_objective
            best_solution = solution
            best_objective = solution["totalValue"]
        end
    end
    return best_solution
end
best_solution = get_best_solution(model)

Dict{String, Union{Bool, Int64, Set{Int64}}} with 7 entries:
  "item[3]"       => 1
  "item[4]"       => 1
  "item[2]"       => 0
  "totalValue"    => -19
  "total_weight"  => 11
  "item[1]"       => 0
  "-total_weight" => -11

## Visualizing the Solution

The problem is solved, let's visualize the solution obtained!

In [13]:
sol = model.statistics.solutions[1]
n = oftype(1, sqrt(length(sol)))
tableau = Matrix{Int}(undef, n, n)
for i in 1:n, j in 1:n
    tableau[i, j] = sol["puzzle_"*string(i)*", "*string(j)]
end
print(" ")
for k in 1:5*n
    print("-")
end
println()
for i in 1:n
    print("|")
    for j in 1:n
        printstyled(" " * string(tableau[i, j], pad=2) * " ", color=tableau[i, j])
        print("|")
    end
    println()
end
print(" ")
for k in 1:5*n
    print("-")
end

 ---------------


| 03 | 02 | 01 |
| 01 | 03 | 02 |
| 02 | 01 | 03 |
 ---------------